In [1]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.8
# plt.rcParams['lines.markersize'] = 4.0
# plt.rcParams['figure.figsize'] = 20, 10

In [2]:
df1 = pd.read_csv("./csv/nod2-1_processed.csv", engine='python')
df2 = pd.read_csv("./csv/nod2-2_processed.csv", engine='python')
df3 = pd.read_csv("./csv/nod2-3_processed.csv", engine='python')
df4 = pd.read_csv("./csv/other2-1_processed.csv", engine='python')
df5 = pd.read_csv("./csv/other2-2_processed.csv", engine='python')
df6 = pd.read_csv("./csv/other2-3_processed.csv", engine='python')

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)

# trvl_csv_path = "csv/entire.csv"
test_csv_dir = "csv/"

test_csv_paths = [test_csv_dir + "multi_people_0_processed.csv",
                  test_csv_dir + "multi_people_1_processed.csv",
                  test_csv_dir + "multi_people_2_processed.csv",
                  test_csv_dir + "multi_people_3_processed.csv",
                 ]
# trvl_df = pd.read_csv(trvl_csv_path, engine='python')
trvl_df = pd.concat([df1, df2, df4, df5, df3, df6])
test_dfs = [pd.read_csv(test_csv_path, engine='python') for test_csv_path in test_csv_paths]

(31, 38)
(58, 38)
(57, 38)
(81, 38)
(67, 38)
(75, 38)


In [3]:
print("== train/validデータの詳細 ==")
trvl_df['label'].astype(int)
cl0 = (trvl_df['label'] == 0)
cl1 = (trvl_df['label'] == 1)
print(f"cl0: {cl0.sum()}, cl1: {cl1.sum()}")
print(f"cl0: {cl0.sum()/(cl0.sum() + cl1.sum())}, cl1: {cl1.sum()/(cl0.sum() + cl1.sum())}")

trvl_df

== train/validデータの詳細 ==
cl0: 223, cl1: 146
cl0: 0.6043360433604336, cl1: 0.3956639566395664


,mean_x,std_x,mad_x,max_x,min_x,energy_x,entropy_x,iqr_x,range_x,skewness_x,...,entropy_z,iqr_z,range_z,skewness_z,kurtosis_z,frame_in,frame_out,timestamp_in,timestamp_out,label
0,0.103313,0.090222,0.0660,0.273,-0.060,0.018813,3.282907,0.13100,0.333,0.030519,...,3.256259,0.01900,0.044,-0.313366,-0.941373,1.0,32.0,0.000,1.033,1.0
1,0.150000,0.065397,0.0620,0.273,0.049,0.026777,3.230602,0.12550,0.224,0.214725,...,3.376073,0.00825,0.036,-0.612509,0.289567,17.0,48.0,0.533,1.567,1.0
2,0.139531,0.066109,0.0545,0.254,0.001,0.023839,3.326393,0.10500,0.253,-0.220985,...,3.354940,0.00725,0.025,0.068811,-0.110982,33.0,64.0,1.067,2.100,1.0
3,0.185719,0.093650,0.0660,0.379,0.001,0.043262,3.316339,0.12300,0.378,0.200418,...,3.168633,0.01625,0.039,0.497027,-0.983946,49.0,80.0,1.600,2.633,1.0
4,0.250437,0.087917,0.0855,0.396,0.096,0.070448,3.282166,0.15275,0.300,0.107027,...,3.204525,0.01350,0.038,0.365520,-0.838306,65.0,96.0,2.133,3.167,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,-0.056281,0.048374,0.0270,0.050,-0.121,0.005508,3.172448,0.04550,0.171,0.776636,...,3.327885,0.05100,0.108,-0.441098,-0.889628,1153.0,1184.0,38.400,39.433,0.0
71,0.002250,0.037818,0.0220,0.050,-0.060,0.001435,3.204886,0.07150,0.110,-0.577385,...,3.170101,0.10300,0.144,-0.086159,-1.484037,1169.0,1200.0,38.933,39.967,0.0
72,0.034281,0.013008,0.0080,0.051,0.008,0.001344,3.301656,0.01725,0.043,-0.660231,...,3.149270,0.01750,0.093,0.983736,0.145093,1185.0,1216.0,39.467,40.500,0.0
73,0.039656,0.006826,0.0045,0.051,0.028,0.001619,3.242756,0.00925,0.023,-0.250682,...,2.854914,0.01950,0.037,0.905031,-0.782166,1201.0,1232.0,40.000,41.033,0.0


In [4]:
def get_na_list(df):
    """
    欠損値のある列リストを生成
    """
    df = df.replace([np.inf, -np.inf], np.nan)
    return list(df.columns[df.isna().any()])

def drop_na(df, axis:int=0, subset: list=None):
    """
    欠損値のある行or列を削除
    axis=0: 行, axis=1: 列
    subset: 削除したいcolumn名のリスト
    """
    df = df.replace([np.inf, -np.inf], np.nan)
#     print(f"欠損値:\n{df.isnull().sum()}")
    if subset is None:
        dropped_df = df.dropna(axis=axis, how="any")
    else:
        dropped_df = df.dropna(subset=subset)
    return dropped_df


def feature_selection(X, y=None, selected_bool_list: list=None):
    """
    分類に有用な特徴量を選択
    """
    from sklearn.ensemble import RandomForestRegressor
    from boruta import BorutaPy
    
    print(len(X.columns))
    if (selected_bool_list is None) and (y is not None):
        # RandomForestRegressorでBorutaを実行
        rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)
        feat_selector.fit(X.values, y.values)

        # 選択された特徴量を確認
        selected_bool_list = feat_selector.support_
        print('選択された特徴量の数: %d' % np.sum(selected_bool_list))
        print(selected_bool_list)
        print(X.columns[selected_bool_list])

    print(f"columns: {len(X.columns)}, bool_list: {len(selected_bool_list)}")
    X_selected = X.iloc[:, selected_bool_list]
    return X_selected, selected_bool_list


def get_Xy_from_df(df):
    """
    X: frame_in, frame_out, timestamp_in,...を除いたカラムのデータ
    y: 正解ラベル
    """
    X = df.iloc[:, 1:-5]
    y = df['label'].astype(int)
    return X, y


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

random_state = 0

# train/validデータの前処理
print(f"before drop_na: {len(trvl_df.columns)}")
trvl_df = drop_na(trvl_df, axis=1)  # 欠損値のある列を削除
print(f"after drop_na: {len(trvl_df.columns)}")

X_trvl, y_trvl = get_Xy_from_df(trvl_df)  # 特徴量, ラベルデータを取得
X_trvl, selected_bool_list = feature_selection(X=X_trvl, y=y_trvl)  # 特徴量を選択
feature_columns = X_trvl.columns

# データの標準化処理
sc = StandardScaler()
sc.fit(X_trvl)
X_trvl = sc.transform(X_trvl)

# trainとvalid用に分割
# X_train, X_valid = X_trvl[142:], X_trvl[0:142]
# y_train, y_valid = y_trvl[142:], y_trvl[0:142]
X_train, X_valid = X_trvl[:237], X_trvl[237:]
y_train, y_valid = y_trvl[:237], y_trvl[237:]

print(X_train.shape, X_valid.shape)
print(y_train.shape, y_valid.shape)
print(y_train, y_valid)


# kernel SVMのインスタンスを生成
model = SVC(kernel='rbf', random_state=random_state)

# モデルの学習
model.fit(X_train, y_train)

# trainデータに対する評価
pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, pred_train)
precision_train = precision_score(y_train, pred_train)
recall_train = recall_score(y_train, pred_train)
print('トレーニングデータに対する正解率： %.2f' % accuracy_train)
print('トレーニングデータに対する適合率： %.2f' % precision_train)
print('トレーニングデータに対する再現率： %.2f' % recall_train)
cm = confusion_matrix(y_train, pred_train)
print(cm)

before drop_na: 38
after drop_na: 38
32
選択された特徴量の数: 6
[False  True  True False False False  True  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False  True]
Index(['mad_x', 'max_x', 'iqr_x', 'range_x', 'min_y', 'kurtosis_z'], dtype='object')
columns: 32, bool_list: 32
(237, 6) (132, 6)
(237,) (132,)
0     1
1     1
2     1
3     1
4     1
     ..
62    0
63    0
64    0
65    0
66    0
Name: label, Length: 237, dtype: int64 0     1
1     1
2     1
3     1
4     1
     ..
70    0
71    0
72    0
73    0
74    0
Name: label, Length: 132, dtype: int64
トレーニングデータに対する正解率： 0.98
トレーニングデータに対する適合率： 0.97
トレーニングデータに対する再現率： 0.99
[[145   3]
 [  1  88]]


In [6]:
# validデータに対する評価
pred_valid = model.predict(X_valid)
accuracy_valid = accuracy_score(y_valid, pred_valid)
precision_valid = precision_score(y_valid, pred_valid)
recall_valid = recall_score(y_valid, pred_valid)
print(f"validデータに対する正解率： {accuracy_valid: .2f}")
print(f"validデータに対する適合率： {precision_valid: .2f}")
print(f"validデータに対する再現率： {recall_valid: .2f}")

cm = confusion_matrix(y_valid, pred_valid)
print(cm)

validデータに対する正解率：  0.92
validデータに対する適合率：  0.93
validデータに対する再現率：  0.89
[[71  4]
 [ 6 51]]


In [7]:
def output_csv(
    base_df: pd.core.frame.DataFrame,
    pred_result: np.ndarray,
    output_path: str="output_pred.csv",
    mode: str="test"
):
    """
    base_df: predに対応するDataFrame
    pred_result: 判別後の予測結果 (ndarray)
    """
    if mode == "test":
        in_columns = ['frame_in', 'frame_out']
        out_columns = ['frame', 'pred']
    else:
        in_columns = ['frame_in', 'frame_out', 'label']
        out_columns = ['frame', 'label', 'pred']
    
    base_df = base_df[in_columns]
    base_df = base_df.assign(pred=pred_result)
    
    frame_in = base_df['frame_in'].iloc[0].astype(int)
    frame_out = base_df['frame_out'].iloc[-1].astype(int)
    cum_pred = np.zeros(frame_out - frame_in)
    


    out_df = pd.DataFrame(columns=out_columns)

    for i, (_, row) in enumerate(base_df.iterrows()):
        fin = row['frame_in'].astype(int)
        fout = row['frame_out'].astype(int)

        for frame in range(fin, fout + 1):
            if (out_df['frame'] == frame).sum() == 0:
                if mode == "test":
                    out_df = out_df.append({
                        'frame': frame,
                        'pred': row['pred'],
                    }, ignore_index=True)
                else:
                    out_df = out_df.append({
                        'frame': frame,
                        'label': row['label'],
                        'pred': row['pred'],
                    }, ignore_index=True)
            else:
                if row['pred'] == 1:
                    out_df.loc[out_df['frame'] == frame, 'pred'] += 1
                    
    if output_path is not None:
        out_df.to_csv(output_path, index=False)

    return out_df

In [8]:
for i, test_df in enumerate(test_dfs):
    # テストデータに対する前処理
    test_df = drop_na(test_df, axis=0)  # 欠損のある行を削除
    X_test = test_df.loc[:, feature_columns]
    X_test = sc.transform(X_test)

    # テストデータに対する精度
    pred_test = model.predict(X_test)    
    output_csv(test_df, pred_test, output_path=f"output/pred_multi_id{i}.csv")

In [26]:
face_df = pd.read_csv(test_csv_dir + "multi_people_1.csv")
face_df_frame = face_df[face_df["frame"] == 6]
face_df_frame["x_30"]

5    615.2
Name: x_30, dtype: float64

In [32]:
import cv2
import tqdm
from typing import Tuple


def get_video_capture(video_path: str) -> cv2.VideoCapture:
    return cv2.VideoCapture(video_path)


def get_video_length(video_path: str) -> int:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))


def get_frame_position(video_capture: cv2.VideoCapture) -> int:
    return int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))


def set_frame_position(video_capture: cv2.VideoCapture, position: int) -> int:
    return int(video_capture.set(cv2.CAP_PROP_POS_FRAMES, position))


def get_video_dimension(video_path: str) -> Tuple[int, int]:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))


def get_video_framerate(video_path: str) -> float:
    video_capture = get_video_capture(video_path)
    return float(video_capture.get(cv2.CAP_PROP_FPS))


def output_video_nod(nod_df, face_df, video_path: str, output_path: str = "output_video_nod"):
    video_capture = get_video_capture(video_path)
    video_length = get_video_length(video_path)

    print(nod_df)
    start, end = nod_df['frame'].iloc[0].astype(int), nod_df['frame'].iloc[-1].astype(int)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    video_writer = cv2.VideoWriter(output_path, fourcc, get_video_framerate(video_path),
                                   get_video_dimension(video_path))

    frame_index = start
    set_frame_position(video_capture, start)  # Move position
    while video_capture.isOpened() and get_frame_position(video_capture) in range(start, end + 1):
        ret, frame = video_capture.read()
        
        nod = nod_df[nod_df['frame'] == frame_index]['pred'].values[0]
        texts = {0: "No", 1: "Nod", 2: "Nod!!!"}

        # Draw circle when nod flag == 1 or 2
        if nod > 0:
            face_df_frame = face_df.iloc[frame_index]
            mid = (int(face_df_frame['x_30']), int(face_df_frame['y_30']))
            radius = int(abs(face_df_frame['y_8'] - face_df_frame['y_30']))
            frame = cv2.circle(frame, mid, radius=radius, color=(0, 0, 255), thickness=int(1 * nod))

            text = texts[nod]
            font = cv2.FONT_HERSHEY_PLAIN
            textsize = cv2.getTextSize(text, font, 1, 2)[0]
            coord = (mid[0] - int(textsize[0] * 1.7), mid[1] - int(radius * 2))
            frame = cv2.putText(frame, text, coord, font, 4, (0, 0, 255), 5, cv2.LINE_AA)

        video_writer.write(frame)
        frame_index += 1

    video_capture.release()
    video_writer.release()
    

def output_video_nod_multi(nod_df, face_id, face_df, video_path: str, output_path: str = "output_video_nod"):
    video_capture = get_video_capture(video_path)
    video_length = get_video_length(video_path)
    
    print(nod_df)
    
    # 動作確認用
    coord_list = [(50, 100), (150, 100), (250, 100), (350, 100)]

    start, end = nod_df['frame'].iloc[0].astype(int), nod_df['frame'].iloc[-1].astype(int)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    video_writer = cv2.VideoWriter(output_path, fourcc, get_video_framerate(video_path),
                                   get_video_dimension(video_path))

    frame_index = start
    set_frame_position(video_capture, start)  # Move position
    while video_capture.isOpened() and get_frame_position(video_capture) in range(start, end + 1):
        
        ret, frame = video_capture.read()
        
        nod = nod_df[nod_df['frame'] == frame_index]['pred'].values[0]
        texts = {0: "No", 1: "Nod", 2: "Nod!!!"}
        
#         face_df_frame = face_df[face_df["frame"] == frame_index]
#         if face_df_frame.shape[0] == 1:
#             mid = (int(face_df_frame['x_30']), int(face_df_frame['y_30']))
#             radius = int(abs(face_df_frame['y_8'] - face_df_frame['y_30']))
#             frame = cv2.circle(frame, mid, radius=radius, color=(0, 0, 255), thickness=2)

        # Draw circle when nod flag == 1 or 2
        if nod > 0:
            face_df_frame = face_df[face_df["frame"] == frame_index]
            if face_df_frame.shape[0] == 1:
                mid = (int(face_df_frame['x_30']), int(face_df_frame['y_30']))
                radius = int(abs(face_df_frame['y_8'] - face_df_frame['y_30']))
                frame = cv2.circle(frame, mid, radius=radius, color=(0, 0, 255), thickness=int(1 * nod))

            text = texts[nod]
            font = cv2.FONT_HERSHEY_PLAIN
            textsize = cv2.getTextSize(text, font, 1, 2)[0]
#             coord = (mid[0] - int(textsize[0] * 1.7), mid[1] - int(radius * 2))
            coord = coord_list[face_id]
            frame = cv2.putText(frame, text, coord, font, 4, (0, 0, 255), 5, cv2.LINE_AA)

        video_writer.write(frame)
        frame_index += 1

    video_capture.release()
    video_writer.release()


video_path = "./output/test.mp4"
face_paths = [test_csv_dir + "multi_people_0.csv",
              test_csv_dir + "multi_people_1.csv",
              test_csv_dir + "multi_people_2.csv",
              test_csv_dir + "multi_people_3.csv"]

for i, face_csv_path in enumerate(face_paths):
    nod_df = pd.read_csv(f"output/pred_multi_id{i}.csv")
    face_df = pd.read_csv(face_csv_path)
    output_path = f"output/pred_test_id{i}.mp4"
    output_video_nod_multi(nod_df, i, face_df, video_path, output_path)

     frame  pred
0      1.0   1.0
1      2.0   1.0
2      3.0   1.0
3      4.0   1.0
4      5.0   1.0
..     ...   ...
517  518.0   0.0
518  519.0   0.0
519  520.0   0.0
520  521.0   0.0
521  522.0   0.0

[522 rows x 2 columns]
     frame  pred
0      1.0   1.0
1      2.0   1.0
2      3.0   1.0
3      4.0   1.0
4      5.0   1.0
..     ...   ...
507  508.0   1.0
508  509.0   1.0
509  510.0   1.0
510  511.0   1.0
511  512.0   1.0

[512 rows x 2 columns]
     frame  pred
0      1.0   1.0
1      2.0   1.0
2      3.0   1.0
3      4.0   1.0
4      5.0   1.0
..     ...   ...
515  516.0   0.0
516  517.0   0.0
517  518.0   0.0
518  519.0   0.0
519  520.0   0.0

[520 rows x 2 columns]
     frame  pred
0      1.0   0.0
1      2.0   0.0
2      3.0   0.0
3      4.0   0.0
4      5.0   0.0
..     ...   ...
507  508.0   0.0
508  509.0   0.0
509  510.0   0.0
510  511.0   0.0
511  512.0   0.0

[512 rows x 2 columns]
